In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
processed_df = pd.read_pickle("data/processed_recipes.pkl")
processed_df

,name,ingredients,description,instructions,cuisine,course,diet,prep_time,categorized_prep_time,cleaned_ingredients,allergens,processed_name,processed_instructions,processed_description,allergen_type,processed_ingredients,combined_all_contexts,combined_name_ingredients
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,"4 cups Indian borage (Doddapatre) , cleaned an...",Doddapatre Tambuli Recipe is a popular healing...,"To begin making the Doddapatre Tambuli, first ...",Indian,Side Dish,Vegetarian,30.0,~30 min,"[indian borage doddapatre, curd dahi yogurt, c...",[yogurt],doddapatre tambuli karpooravalli thayir pachadi,doddapatre tambuli doddapatre leave dry towel ...,doddapatre tambuli popular healing cold yoghur...,[Dairy],indian borage doddapatre curd dahi yogurt cumi...,doddapatre tambuli karpooravalli thayir pachad...,doddapatre tambuli karpooravalli thayir pachad...
1,Fish Tandoori Recipe,"1 Fish , nicely washed with bones on (I used P...",Have you ever tried out fish tandoori? Trust m...,"To begin making the Fish Tandoori recipe, keep...",North Indian,Appetizer,High Protein Non Vegetarian,80.0,~1 hour 30 min,"[fish, curd dahi yogurt, onion, green chilli, ...",[fish],fish tandoori,fish tandoori head fin inside fish many horizo...,fish tandoori delicious chicken tandoori tando...,[Fish and Shellfish],fish curd dahi yogurt onion green chilli tando...,fish tandoori fish curd dahi yogurt onion gree...,fish tandoori fish curd dahi yogurt onion gree...
2,Arbi Shimla Mirch Sabzi Recipe – Colocasia Cap...,300 grams Colocasia root (Arbi) 1 Green Bell ...,Arbi Shimla Mirch Sabzi Recipe is a variation ...,To begin making the Arbi Shimla Mirch Sabzi Re...,North Indian,Main Course,No Onion No Garlic (Sattvic),55.0,~1 hour,"[colocasia root arbi, green bell pepper capsic...",[peanut],arbi shimla mirch sabzi colocasia capsicum sabzi,arbi shimla mirch sabzi arbi pressure pressure...,arbi shimla mirch sabzi variation aloo capsicu...,"[Tree Nuts and Peanuts, Legumes and Seeds]",colocasia root arbi green bell pepper capsicum...,arbi shimla mirch sabzi colocasia capsicum sab...,arbi shimla mirch sabzi colocasia capsicum sab...
3,Ambur Style Brinjal Curry Recipe,5 Brinjal (Baingan / Eggplant) 1 Onion 2 Tom...,Brinjal Curry is prepared in different ways in...,To begin making the Ambur Style Brinjal Curry ...,Indian,Lunch,Vegetarian,20.0,~30 min,"[brinjal baingan eggplant, onion, ginger garli...","[sesame, peanut, mustard]",ambur style brinjal curry,ambur style brinjal curry oil heavy bottomed o...,brinjal curry different way different region s...,"[Legumes and Seeds, Tree Nuts and Peanuts]",brinjal baingan eggplant onion ginger garlic p...,ambur style brinjal curry brinjal baingan eggp...,ambur style brinjal curry brinjal baingan eggp...
4,Mavinakayi Menasinakai Curry Recipe - Raw Man...,"1 cup Mango (Raw) , diced 2 tablespoons Jagge...",Mangalorean Mavinakayi Menasinakai Curry Recip...,To begin making Mangalorean Mavinakayi Menasin...,Mangalorean,Side Dish,Vegetarian,35.0,~45 min,"[mango raw, jaggery, turmeric powder haldi, ta...",[sesame],mavinakayi menasinakai curry raw mango coconut...,mangalorean mavinakayi menasinakai curry raw m...,mangalorean mavinakayi menasinakai curry raw m...,[Legumes and Seeds],mango raw jaggery turmeric powder haldi tamari...,mavinakayi menasinakai curry raw mango coconut...,mavinakayi menasinakai curry raw mango coconut...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,Spinach Oats Vada Recipe,"Spinach Leaves (Palak), Chana dal (Bengal Gram...",Spinach Oats Vada Recipe is a healthy twist to...,"To prepare Spinach Oats Vada Recipe, in a larg...",South Indian,Snack,Vegetarian,50.0,~1 hour,"[spinach leave palak, chana dal bengal gram da...",[oats],spinach oats vada,spinach oats vada large mixing ingredient vada...,spinach oats vada healthy twist vada superfood...,[Grains and Gluten],spinach leave palak chana dal bengal gram dal ...,spinach oats vada spinach leave palak chana da...,sp

Train the recommendation model 

In [3]:
def get_recommendations(dish_name, df, best_num_topics, vectorizer, model, num_recommendations=5):
    vectorizer_matrix =  vectorizer.fit_transform(processed_df['processed_name'])

    # Train the model with the optimal number of topics
    called_model = model(n_components=best_num_topics, random_state=42)
    called_model.fit(vectorizer_matrix)
    topic_matrix = called_model.transform(vectorizer_matrix)# Topic distribution for each recipe

    # Add topic distributions to df
    for i in range(best_num_topics):
        processed_df[f'topic_{i}'] = topic_matrix[:, i]

    # Find the index of the given dish name
    recipe_index = df[df['name'].str.contains(dish_name, case=False, na=False)].index
    if not recipe_index.any():
        print("Dish not found.")
        return None
    
    recipe_index = recipe_index[0]  # Get the first match if there are multiple
    
    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    similar_indices = cosine_similarities[recipe_index].argsort()[-num_recommendations-1:-1][::-1]
    
    # Get the recommended recipes
    recommended_recipes = df.iloc[similar_indices].copy()
    recommended_recipes[ "similarity_score"] = cosine_similarities[recipe_index][similar_indices]

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergen_type', 'categorized_prep_time']]

Get recipe recommendation using best NMF model on TF-IDF vectorizer

In [4]:
recommended_recipes_nmf_tfidf = get_recommendations("dosa", processed_df, 8, TfidfVectorizer(), NMF, 5)  
recommended_recipes_nmf_tfidf

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5402,Vazhaipoo Bajra Adai Recipe - Banana Flower & ...,1.000000,"[banana flower vazhaipoo, onion, turmeric powd...",Tamil Nadu,South Indian Breakfast,Diabetic Friendly,[Dairy],~3.0 hours 30 min
4178,Foxtail Millet Podi Dosa Recipe,0.989098,"[foxtail millet, idli rice, white urad dal spl...",South Indian,South Indian Breakfast,Diabetic Friendly,Unknown,~9.0 hours 30 min
2547,Foxtail Millet Podi Dosa Recipe,0.989098,"[foxtail millet, idli rice, white urad dal spl...",South Indian,South Indian Breakfast,Diabetic Friendly,Unknown,~9.0 hours 30 min
5864,Thecha Paniyaram Recipe,0.987947,"[idli dosa batter, sooji rava, coriander dhani...",Maharashtrian,Snack,Vegetarian,"[Tree Nuts and Peanuts, Legumes and Seeds]",~30 min
9066,Homemade Italian Calzones Recipe,0.982858,"[lukewarm water, active dry yeast, sugar, extr...",Italian,Snack,Vegetarian,[Dairy],~15 min


Get recipe recommendation using best SVD model on TF-IDF vectorizer

In [5]:
recommended_recipes_svd_tfidf = get_recommendations("dosa", processed_df, 24, TfidfVectorizer(), TruncatedSVD, 5)  
recommended_recipes_svd_tfidf

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5402,Vazhaipoo Bajra Adai Recipe - Banana Flower & ...,1.000000,"[banana flower vazhaipoo, onion, turmeric powd...",Tamil Nadu,South Indian Breakfast,Diabetic Friendly,[Dairy],~3.0 hours 30 min
2511,Oatmeal Dosa Recipe - Healthy Instant Oats Dos...,0.953593,"[instant oat oatmeal, whole wheat flour, sooji...",South Indian,South Indian Breakfast,High Protein Vegetarian,[Grains and Gluten],~1 hour
8333,Oatmeal Dosa Recipe - Healthy Instant Oats Dos...,0.953593,"[instant oat oatmeal, whole wheat flour, sooji...",South Indian,South Indian Breakfast,High Protein Vegetarian,[Grains and Gluten],~1 hour
2510,Javvarisi Dosa Recipe - Soft & Instant Sabudan...,0.939886,"[sabudana tapioca pearl, curd dahi yogurt, idl...",South Indian,South Indian Breakfast,Vegetarian,"[Dairy, Legumes and Seeds]",~1 hour
8332,Javvarisi Dosa Recipe - Soft & Instant Sabudan...,0.939886,"[sabudana tapioca pearl, curd dahi yogurt, idl...",South Indian,South Indian Breakfast,Vegetarian,"[Dairy, Legumes and Seeds]",~1 hour


Get recipe recommendation using best NMF model on Count vectorizer

In [6]:
recommended_recipes_nmf_count = get_recommendations("dosa", processed_df, 15, CountVectorizer(), NMF, 5)  
recommended_recipes_nmf_count

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5402,Vazhaipoo Bajra Adai Recipe - Banana Flower & ...,1.000000,"[banana flower vazhaipoo, onion, turmeric powd...",Tamil Nadu,South Indian Breakfast,Diabetic Friendly,[Dairy],~3.0 hours 30 min
8530,Whole Wheat Waffles with Caramelised Bananas (...,0.940663,"[flaxmeal egg replacer, milk, oil, sugar, powd...",Continental,World Breakfast,Vegetarian,"[Dairy, Grains and Gluten, Eggs]",~1 hour
4467,Homemade Whole Wheat Pav Recipe / Ladi Pav,0.932641,"[whole wheat flour, purpose flour maida, activ...",Goan,Side Dish,Vegetarian,"[Dairy, Grains and Gluten]",~1 hour 45 min
290,Whole Wheat Soya Keema Kulchas,0.922455,"[whole wheat flour, curd dahi yogurt, milk, aj...",North Indian,Dinner,High Protein Vegetarian,"[Dairy, Legumes and Seeds, Grains and Gluten]",~3.0 hours 30 min
5154,Whole Wheat Soya Keema Kulchas,0.922455,"[whole wheat flour, curd dahi yogurt, milk, aj...",North Indian,Dinner,High Protein Vegetarian,"[Dairy, Legumes and Seeds, Grains and Gluten]",~3.0 hours 30 min


Get recipe recommendation using best SVD model on Count vectorizer

In [7]:
recommended_recipes_svd_count = get_recommendations("dosa", processed_df, 3, CountVectorizer(), TruncatedSVD, 5)  
recommended_recipes_svd_count

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
561,Vazhaipoo Bajra Adai Recipe - Banana Flower & ...,1.000000,"[banana flower vazhaipoo, onion, turmeric powd...",Tamil Nadu,South Indian Breakfast,Diabetic Friendly,[Dairy],~3.0 hours 30 min
4427,Pull Apart Bread With Basil Pesto Recipe,0.999951,"[whole wheat flour, instant oat oatmeal, extra...",Continental,Appetizer,Eggetarian,"[Grains and Gluten, Eggs, Dairy]",~1 hour 45 min
6770,Gooseberry Ginger Detox Health Drink Recipe,0.999917,"[amla gooseberry, ginger, lemon, black, salt k...",Indian,Snack,Vegetarian,Unknown,~1 hour
3929,Russian Radish Cucumber Salad Recipe,0.999910,"[red radish, mullangi radish, cucumber, garlic...",Continental,Side Dish,Eggetarian,"[Grains and Gluten, Dairy, Eggs]",~30 min
5059,"Summer Salad Recipe-Red Cabbage, Lettuce Onion...",0.999849,"[red cabbage, iceberg lettuce, onion, salt, le...",Continental,Appetizer,Vegetarian,[Dairy],~15 min


Filter the recommended recipes based on user preferences


In [8]:
# Define a function to filter recipes based on user preferences
def filter_recipes(df, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None):
    filtered_df = df
    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]
    return filtered_df

In [9]:
# Example 
recommended_recipes = recommended_recipes_nmf_tfidf
if recommended_recipes is not None:
    filtered_recommendations = filter_recipes(recommended_recipes, diet = "Vegetarian")
    display(filtered_recommendations)

,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergen_type,categorized_prep_time
5864,Thecha Paniyaram Recipe,0.987947,"[idli dosa batter, sooji rava, coriander dhani...",Maharashtrian,Snack,Vegetarian,"[Tree Nuts and Peanuts, Legumes and Seeds]",~30 min
9066,Homemade Italian Calzones Recipe,0.982858,"[lukewarm water, active dry yeast, sugar, extr...",Italian,Snack,Vegetarian,[Dairy],~15 min
